# Quantum Fourier Transform



In [ ]:
from mpqp.gates import H, X, Ry, CRk
from mpqp import QCircuit
from mpqp.measures import BasisMeasure
from mpqp.execution.devices import IBMDevice
from mpqp.execution import run
from mpqp.tools.visualization import plot_results_sample_mode
import matplotlib.pyplot as plt

In [16]:
class QFT(QCircuit):
      
    def __init__(self,n_qubits,inverse=False):
        
        super().__init__(n_qubits, nb_cbits=n_qubits)
        self.inverse = inverse
        self.apply()
    
    def apply(self):
        # [self.add([H(j),[CRk(i,i,j) for i in range(j+1,n_qubits-1)]] for j in range(0,n_qubits))]
        
        (self.add([H(j),[CRk(i+1,i,j) for i in range(j+1,self.nb_qubits)],Barrier()] for j in range(0,self.nb_qubits)))
        self.add([SWAP(i,self.nb_qubits-1-i) for i in range(0,int(floor(self.nb_qubits/2)))])
        self.add(BasisMeasure([i for i in range(self.nb_qubits)], shots=0))
        
        if self.inverse == True:
           self.inverse() 

In [17]:
qc = QFT(n_qubits=5)
print(qc)

      ┌───┐                                          ░                »
 q_0: ┤ H ├─■─────────■─────────■─────────■──────────░────────────────»
      └───┘ │P(pi/2)  │         │         │          ░ ┌───┐          »
 q_1: ──────■─────────┼─────────┼─────────┼──────────░─┤ H ├─■────────»
                      │P(pi/4)  │         │          ░ └───┘ │P(pi/4) »
 q_2: ────────────────■─────────┼─────────┼──────────░───────■────────»
                                │P(pi/8)  │          ░                »
 q_3: ──────────────────────────■─────────┼──────────░────────────────»
                                          │P(pi/16)  ░                »
 q_4: ────────────────────────────────────■──────────░────────────────»
                                                     ░                »
c: 10/════════════════════════════════════════════════════════════════»
                                                                      »
«                            ░                            ░     

In [9]:
results = run(qc, [IBMDevice.AER_SIMULATOR_STATEVECTOR,ATOSDevice.MYQLM_PYLINALG]) #IBMDevice.AER_SIMULATOR
print(results)

# plot_results_sample_mode(results)
# print(qc)
# plt.show()


BatchResult: 2 results
Result: IBMDevice, AER_SIMULATOR_STATEVECTOR

        State vector: [0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j
 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j
 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j
 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j
 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j
 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j 0.1767767+0.j
 0.1767767+0.j 0.1767767+0.j]
        Probabilities: [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125]
        Number of qubits: 5

Result: ATOSDevice, MYQLM_PYLINALG

        State vector: [0.17677669+0.00000000e+00j 0.17677669+0.00000000e+00j
 0.17677669+0.00000000e+00j

In [18]:
qc1 = QCircuit([H(1)])
results1 = run(qc1, [IBMDevice.AER_SIMULATOR_STATEVECTOR
                    #  ,ATOSDevice.MYQLM_PYLINALG
                     ]) #IBMDevice.AER_SIMULATOR
print(results1)


BatchResult: 1 results
Result: IBMDevice, AER_SIMULATOR_STATEVECTOR

        State vector: [0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
        Probabilities: [0.5 0.5 0.  0. ]
        Number of qubits: 2




In [19]:
qc2=qc1+(QFT(2))
print(qc2)
results2 = run(qc2, [IBMDevice.AER_SIMULATOR_STATEVECTOR
                    #  ,ATOSDevice.MYQLM_PYLINALG
                     ]) #IBMDevice.AER_SIMULATOR
print(results2)


     ┌───┐           ░       ░    ┌─┐   
q_0: ┤ H ├─■─────────░───────░──X─┤M├───
     ├───┤ │P(pi/2)  ░ ┌───┐ ░  │ └╥┘┌─┐
q_1: ┤ H ├─■─────────░─┤ H ├─░──X──╫─┤M├
     └───┘           ░ └───┘ ░     ║ └╥┘
c: 2/══════════════════════════════╩══╩═
                                   0  1 
BatchResult: 1 results
Result: IBMDevice, AER_SIMULATOR_STATEVECTOR

        State vector: [0.70710678-4.32978028e-17j 0.35355339+3.53553391e-01j
 0.        +4.32978028e-17j 0.35355339-3.53553391e-01j]
        Probabilities: [5.00000000e-01 2.50000000e-01 1.87469973e-33 2.50000000e-01]
        Number of qubits: 2




In [25]:
qc3 = QCircuit([Ry(2*np.arccos(1/np.sqrt(3)),0),Ry(np.pi/4,1),CZ(0,1),Ry(-np.pi/4,1),CNOT(0,1),CNOT(1,2),X(0)])
print(qc3)
results3 = run(qc3, [IBMDevice.AER_SIMULATOR_STATEVECTOR
                    #  ,ATOSDevice.MYQLM_PYLINALG
                     ]) #IBMDevice.AER_SIMULATOR
print(results3)

     ┌────────────┐                    ┌───┐
q_0: ┤ Ry(1.9106) ├─■───────────────■──┤ X ├
     └┬─────────┬─┘ │ ┌──────────┐┌─┴─┐└───┘
q_1: ─┤ Ry(π/4) ├───■─┤ Ry(-π/4) ├┤ X ├──■──
      └─────────┘     └──────────┘└───┘┌─┴─┐
q_2: ──────────────────────────────────┤ X ├
                                       └───┘
BatchResult: 1 results
Result: IBMDevice, AER_SIMULATOR_STATEVECTOR

        State vector: [-5.77350269e-01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  5.77350269e-01+0.j  5.77350269e-01+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  7.84807271e-18+0.j]
        Probabilities: [3.33333333e-01 0.00000000e+00 0.00000000e+00 3.33333333e-01
 3.33333333e-01 0.00000000e+00 0.00000000e+00 6.15922453e-35]
        Number of qubits: 3




In [26]:
qc4 = qc3+QFT(3)
results4 = run(qc4, [IBMDevice.AER_SIMULATOR_STATEVECTOR
                    #  ,ATOSDevice.MYQLM_PYLINALG
                     ]) #IBMDevice.AER_SIMULATOR
print(results4)

BatchResult: 1 results
Result: IBMDevice, AER_SIMULATOR_STATEVECTOR

        State vector: [ 0.20412415-7.49939943e-17j -0.55258586+1.44337567e-01j
 -0.14433757-1.44337567e-01j -0.20412415-1.96261557e-17j
 -0.20412415+2.49979981e-17j -0.26391072-1.44337567e-01j
  0.14433757+1.44337567e-01j -0.61237244+6.96221520e-17j]
        Probabilities: [0.04166667 0.32618446 0.04166667 0.04166667 0.04166667 0.0904822
 0.04166667 0.375     ]
        Number of qubits: 3


